### Import libraries

In [ ]:
!pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np

# Oversampling
from imblearn.over_sampling import RandomOverSampler

# Optimisation
from bayes_opt import BayesianOptimization

# Model
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Input, Embedding, Dense, LSTM, concatenate, Dropout, LeakyReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras import metrics
from keras.losses import BinaryCrossentropy
from keras.wrappers.scikit_learn import KerasClassifier

# Evaluate
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

Mounted at /content/gdrive


In [ ]:
X_train = pd.read_csv('../datasets/word2vec/train_data_imputed_FINAL.csv')
X_test = pd.read_csv('../datasets/word2vec/test_data_imputed_FINAL.csv')
y_train = pd.read_csv('../datasets/word2vec/y_train_FINAL.csv').iloc[:,1:]
y_test = pd.read_csv('../datasets/word2vec/y_test_FINAL.csv').iloc[:,1:]

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(11516, 263)
(11516, 1)
(2880, 263)
(2880, 1)


### Oversampling

In [ ]:
# Before oversampling
y_train.value_counts()

fraudulent
0             11293
1               223
dtype: int64

In [ ]:
ros = RandomOverSampler(sampling_strategy=1, random_state=42)
X_train_over, y_train_over = ros.fit_resample(X_train, y_train)

In [ ]:
X_train_over = X_train_over.values
X_train_over = X_train_over.reshape(X_train_over.shape[0], 1, X_train_over.shape[1])

X_test = X_test.values
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [ ]:
# After oversampling
y_train_over.value_counts()

fraudulent
0             11293
1             11293
dtype: int64

### LSTM

In [ ]:
def eval(model):
    loss_train, accuracy_train, precision_train, recall_train, auc_train = model.evaluate(X_train_over, y_train_over, verbose=0)
    f1_train = 2* precision_train * recall_train / (precision_train + recall_train)
    train_res = [loss_train, accuracy_train, precision_train, recall_train, auc_train, f1_train]

    loss , accuracy , precision , recall, auc = model.evaluate(X_test, y_test, verbose=0)
    f1 = 2* precision * recall / (precision + recall)
    test_res = [loss , accuracy , precision , recall, auc, f1]

    return pd.DataFrame([train_res, test_res], index=['Train', 'Test'], columns=['Loss', 'Accuracy', 'Precision', 'Recall', 'AUC', 'F1'])

In [ ]:
model = Sequential()
model.add(LSTM(128))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy', 'Precision', 'Recall', 'AUC'])

In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
history = model.fit(X_train_over,y_train_over,epochs=20,batch_size=64, callbacks=[es])

Epoch 1/20
353/353 [==============================] - 6s 7ms/step - loss: 0.2677 - accuracy: 0.8833 - precision: 0.8577 - recall: 0.9191 - auc: 0.9577
Epoch 2/20
353/353 [==============================] - 2s 7ms/step - loss: 0.0602 - accuracy: 0.9812 - precision: 0.9713 - recall: 0.9917 - auc: 0.9964
Epoch 3/20
353/353 [==============================] - 2s 7ms/step - loss: 0.0275 - accuracy: 0.9920 - precision: 0.9875 - recall: 0.9966 - auc: 0.9988
Epoch 4/20
353/353 [==============================] - 2s 7ms/step - loss: 0.0154 - accuracy: 0.9958 - precision: 0.9936 - recall: 0.9981 - auc: 0.9995
Epoch 5/20
353/353 [==============================] - 2s 7ms/step - loss: 0.0183 - accuracy: 0.9943 - precision: 0.9922 - recall: 0.9964 - auc: 0.9994
Epoch 6/20
353/353 [==============================] - 2s 7ms/step - loss: 0.0050 - accuracy: 0.9986 - precision: 0.9977 - recall: 0.9995 - auc: 0.9998
Epoch 7/20
353/353 [==============================] - 2s 7ms/step - loss: 0.0031 - accuracy: 0

In [ ]:
eval(model)

,Loss,Accuracy,Precision,Recall,AUC,F1
Train,0.000004,1.000000,1.000000,1.000000,1.000000,1.00000
Test,0.061871,0.990278,0.804348,0.660714,0.908795,0.72549


### Hyperparameter tuning

Diff #layers, same #neurons each

In [ ]:
LeakyReLU = LeakyReLU(alpha=0.1)

In [ ]:
def lstm_tuner(batch_size, neurons, activation, optimizer, dropout, dropout_rate, learning_rate, epochs, layers):

    optimizerL = ['Adam', 'RMSprop']
    optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'RMSprop':RMSprop(learning_rate=learning_rate)}
    activationL = ['relu', 'sigmoid', 'tanh', 'selu', 'elu', LeakyReLU]
    neurons = round(neurons)
    activation = activationL[round(activation)]
    optimizer = optimizerD[optimizerL[round(optimizer)]]
    batch_size = round(batch_size)
    epochs = round(epochs)
    layers = round(layers)

    def create_model():
      # for reproducibility 
      np.random.seed(42)
      tf.random.set_seed(42)

      tuned_model = Sequential()
      tuned_model.add(LSTM(neurons, return_sequences=False)) 
      if dropout > 0.5:
        tuned_model.add(Dropout(dropout_rate, seed=42))

      for i in range(layers): #varying first set of hidden layers
        if activationL == 'LeakyReLU':
          tuned_model.add(Dense(neurons, activation=activation))
          tuned_model.add(LeakyReLU(alpha=0.1))
        else:
          tuned_model.add(Dense(neurons, activation=activation))
        if dropout > 0.5:
          tuned_model.add(Dropout(dropout_rate, seed=42))

      tuned_model.add(Dense(1, activation='sigmoid')) #output layer
      tuned_model.compile(loss='binary_crossentropy', optimizer = optimizer, metrics=['accuracy', 'Precision', 'Recall', 'AUC'])

      return tuned_model
    
    es = EarlyStopping(monitor="loss", patience=3)
    mc = ModelCheckpoint('Best_tuned_LSTM_.h5', monitor='val_loss',mode='min',verbose=2, save_best_only=True)

    tuned_lstm = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose=1) 
    score = cross_val_score(tuned_lstm, X_train_over, y_train_over, fit_params={'callbacks':[es]}, error_score='raise', scoring='f1').mean() #5 fold cross validation default
    return score

In [ ]:
# Parameters to search
params ={
    'neurons': (16, 128),
    'activation':(0, 5),
    'optimizer':(0, 1),
    'learning_rate':(0.0005, 0.001),
    'batch_size':(32, 128),
    'epochs':(15, 30),
    'layers':(0, 5),
    'dropout':(0, 1),
    'dropout_rate':(0, 0.3)
}

In [ ]:
# Run Bayesian Optimization
lstm_bo = BayesianOptimization(lstm_tuner, params, random_state=42)
lstm_bo.maximize(init_points=20, n_iter=5)

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  layers   | learni... |  neurons  | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


Epoch 1/17
147/147 [==============================] - 5s 10ms/step - loss: 0.4653 - accuracy: 0.7880 - precision: 0.7693 - recall: 0.9398 - auc: 0.8452
Epoch 2/17
147/147 [==============================] - 1s 9ms/step - loss: 0.2264 - accuracy: 0.9132 - precision: 0.9115 - recall: 0.9525 - auc: 0.9644
Epoch 3/17
147/147 [==============================] - 1s 10ms/step - loss: 0.1606 - accuracy: 0.9440 - precision: 0.9396 - recall: 0.9722 - auc: 0.9796
Epoch 4/17
147/147 [==============================] - 1s 10ms/step - loss: 0.1295 - accuracy: 0.9554 - precision: 0.9505 - recall: 0.9791 - auc: 0.9853
Epoch 5/17
147/147 [==============================] - 1s 10ms/step - loss: 0.1069 - accuracy: 0.9658 - precision: 0.9596 - recall: 0.9863 - auc: 0.9891
Epoch 6/17
147/147 [==============================] - 1s 10ms/step - loss: 0.0868 - accuracy: 0.9718 - precision: 0.9672 - recall: 0.9880 - auc: 0.9928
Epoch 7/17
147/147 [==============================] - 1s 9ms/step - loss: 0.0720 - accura

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


532/532 [==============================] - 5s 4ms/step - loss: 0.3989 - accuracy: 0.8224 - precision: 0.8068 - recall: 0.9381 - auc: 0.8892
Epoch 2/18
532/532 [==============================] - 2s 4ms/step - loss: 0.1837 - accuracy: 0.9359 - precision: 0.9329 - recall: 0.9661 - auc: 0.9740
Epoch 3/18
532/532 [==============================] - 2s 4ms/step - loss: 0.1299 - accuracy: 0.9572 - precision: 0.9525 - recall: 0.9798 - auc: 0.9851
Epoch 4/18
532/532 [==============================] - 2s 4ms/step - loss: 0.1005 - accuracy: 0.9679 - precision: 0.9648 - recall: 0.9841 - auc: 0.9905
Epoch 5/18
532/532 [==============================] - 2s 4ms/step - loss: 0.0793 - accuracy: 0.9759 - precision: 0.9715 - recall: 0.9901 - auc: 0.9929
Epoch 6/18
532/532 [==============================] - 2s 4ms/step - loss: 0.0648 - accuracy: 0.9787 - precision: 0.9766 - recall: 0.9893 - auc: 0.9955
Epoch 7/18
532/532 [==============================] - 2s 4ms/step - loss: 0.0500 - accuracy: 0.9836 - pre

...

292/292 [==============================] - 6s 8ms/step - loss: 0.3348 - accuracy: 0.8523 - precision: 0.8521 - recall: 0.9217 - auc: 0.9207
Epoch 2/18
292/292 [==============================] - 2s 8ms/step - loss: 0.1679 - accuracy: 0.9373 - precision: 0.9365 - recall: 0.9643 - auc: 0.9778
Epoch 3/18
292/292 [==============================] - 2s 8ms/step - loss: 0.1123 - accuracy: 0.9618 - precision: 0.9611 - recall: 0.9779 - auc: 0.9890
Epoch 4/18
292/292 [==============================] - 2s 8ms/step - loss: 0.0766 - accuracy: 0.9734 - precision: 0.9733 - recall: 0.9842 - auc: 0.9942
Epoch 5/18
292/292 [==============================] - 2s 8ms/step - loss: 0.0531 - accuracy: 0.9812 - precision: 0.9819 - recall: 0.9879 - auc: 0.9969
Epoch 6/18
292/292 [==============================] - 2s 8ms/step - loss: 0.0431 - accuracy: 0.9855 - precision: 0.9860 - recall: 0.9907 - auc: 0.9977
Epoch 7/18
292/292 [==============================] - 2s 8ms/step - loss: 0.0332 - accuracy: 0.9884 - pre

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


307/307 [==============================] - 5s 7ms/step - loss: 0.2720 - accuracy: 0.8819 - precision: 0.8657 - recall: 0.9582 - auc: 0.9475
Epoch 2/20
307/307 [==============================] - 2s 7ms/step - loss: 0.0749 - accuracy: 0.9778 - precision: 0.9726 - recall: 0.9921 - auc: 0.9938
Epoch 3/20
307/307 [==============================] - 2s 7ms/step - loss: 0.0346 - accuracy: 0.9903 - precision: 0.9871 - recall: 0.9973 - auc: 0.9981
Epoch 4/20
307/307 [==============================] - 2s 7ms/step - loss: 0.0173 - accuracy: 0.9951 - precision: 0.9932 - recall: 0.9989 - auc: 0.9993
Epoch 5/20
307/307 [==============================] - 2s 7ms/step - loss: 0.0172 - accuracy: 0.9955 - precision: 0.9948 - recall: 0.9980 - auc: 0.9991
Epoch 6/20
307/307 [==============================] - 2s 6ms/step - loss: 0.0102 - accuracy: 0.9973 - precision: 0.9967 - recall: 0.9989 - auc: 0.9998
Epoch 7/20
307/307 [==============================] - 2s 7ms/step - loss: 0.0033 - accuracy: 0.9990 - pre

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


317/317 [==============================] - 5s 6ms/step - loss: 0.2674 - accuracy: 0.8821 - precision: 0.8651 - recall: 0.9594 - auc: 0.9497
Epoch 2/30
317/317 [==============================] - 2s 6ms/step - loss: 0.0699 - accuracy: 0.9784 - precision: 0.9718 - recall: 0.9939 - auc: 0.9949
Epoch 3/30
317/317 [==============================] - 2s 6ms/step - loss: 0.0330 - accuracy: 0.9900 - precision: 0.9868 - recall: 0.9971 - auc: 0.9987
Epoch 4/30
317/317 [==============================] - 2s 6ms/step - loss: 0.0136 - accuracy: 0.9966 - precision: 0.9949 - recall: 0.9996 - auc: 0.9997
Epoch 5/30
317/317 [==============================] - 2s 6ms/step - loss: 0.0097 - accuracy: 0.9980 - precision: 0.9971 - recall: 0.9996 - auc: 0.9997
Epoch 6/30
317/317 [==============================] - 2s 6ms/step - loss: 0.0131 - accuracy: 0.9957 - precision: 0.9953 - recall: 0.9979 - auc: 0.9996
Epoch 7/30
317/317 [==============================] - 2s 6ms/step - loss: 0.0027 - accuracy: 0.9997 - pre

In [ ]:
# Best parameters
params_lstm_ = lstm_bo.max['params']
params_lstm_

{'activation': 2.1597250932105787,
 'batch_size': 59.95799745901202,
 'dropout': 0.6118528947223795,
 'dropout_rate': 0.04184815819561255,
 'epochs': 19.382169728028273,
 'layers': 1.8318092164684585,
 'learning_rate': 0.000728034992108518,
 'neurons': 103.93970767601752,
 'optimizer': 0.19967378215835974}

In [ ]:
learning_rate = params_lstm_['learning_rate']
activationL = ['relu', 'sigmoid', 'tanh', 'selu', 'elu', LeakyReLU]
activation= activationL[round(params_lstm_['activation'])]
batch_size = round(params_lstm_['batch_size'])
epochs = round(params_lstm_['epochs'])
layers = round(params_lstm_['layers'])
dropout =  params_lstm_['dropout']
dropout_rate = params_lstm_['dropout_rate']
neurons = round(params_lstm_['neurons'])
optimizerL = ['Adam', 'RMSprop']
optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'RMSprop':RMSprop(learning_rate=learning_rate)}
optimizer = optimizerD[optimizerL[round(params_lstm_['optimizer'])]]

In [ ]:
# Best model
tuned_model = Sequential()
tuned_model.add(LSTM(neurons, return_sequences=False)) 
if dropout > 0.5:
  tuned_model.add(Dropout(dropout_rate, seed=42))

for i in range(layers): #varying first set of hidden layers
  if activationL == 'LeakyReLU':
    tuned_model.add(Dense(neurons, activation=activation))
    tuned_model.add(LeakyReLU(alpha=0.1))
  else:
    tuned_model.add(Dense(neurons, activation=activation))
  if dropout > 0.5:
    tuned_model.add(Dropout(dropout_rate, seed=42))

tuned_model.add(Dense(1, activation='sigmoid')) #output layer
tuned_model.compile(loss=BinaryCrossentropy(from_logits=False), optimizer = optimizer, metrics=['accuracy', 'Precision', 'Recall', 'AUC'])
es = EarlyStopping(monitor="loss", patience=3)
tuned_model.fit(X_train_over, y_train_over, epochs=epochs, batch_size=batch_size, callbacks=[es])
tuned_model.summary()

Epoch 1/19
377/377 [==============================] - 6s 6ms/step - loss: 0.2407 - accuracy: 0.8972 - precision: 0.8800 - recall: 0.9200 - auc: 0.9640
Epoch 2/19
377/377 [==============================] - 2s 6ms/step - loss: 0.0727 - accuracy: 0.9761 - precision: 0.9679 - recall: 0.9849 - auc: 0.9951
Epoch 3/19
377/377 [==============================] - 2s 6ms/step - loss: 0.0304 - accuracy: 0.9904 - precision: 0.9866 - recall: 0.9943 - auc: 0.9988
Epoch 4/19
377/377 [==============================] - 2s 6ms/step - loss: 0.0218 - accuracy: 0.9927 - precision: 0.9909 - recall: 0.9946 - auc: 0.9993
Epoch 5/19
377/377 [==============================] - 2s 6ms/step - loss: 0.0164 - accuracy: 0.9941 - precision: 0.9927 - recall: 0.9956 - auc: 0.9996
Epoch 6/19
377/377 [==============================] - 2s 6ms/step - loss: 0.0132 - accuracy: 0.9954 - precision: 0.9942 - recall: 0.9965 - auc: 0.9997
Epoch 7/19
377/377 [==============================] - 2s 6ms/step - loss: 0.0080 - accuracy: 0

In [ ]:
# Model performance
eval(tuned_model)

,Loss,Accuracy,Precision,Recall,AUC,F1
Train,0.004056,0.999956,0.999911,1.000000,1.000000,0.999956
Test,0.048240,0.990625,0.808511,0.678571,0.952597,0.737864


### Hyperparameter tuning 2
4 layers, diff #neurons for each

In [ ]:
def lstm_tuner_2(batch_size, neurons1, neurons2, neurons3, neurons4, activation, optimizer, dropout, dropout_rate, learning_rate, epochs):

    optimizerL = ['Adam', 'RMSprop']
    optimizerD= {'Adam':Adam(learning_rate=learning_rate),'RMSprop':RMSprop(learning_rate=learning_rate)}
    activationL = ['relu', 'sigmoid', 'tanh', 'selu', 'elu', LeakyReLU]
    neurons1 = round(neurons1)
    neurons2 = round(neurons2)
    neurons3 = round(neurons3)
    neurons4 = round(neurons4)
    activation = activationL[round(activation)]
    optimizer = optimizerD[optimizerL[round(optimizer)]]
    batch_size = round(batch_size)
    epochs = round(epochs)

    def create_model():
      # for reproducibility 
      np.random.seed(42)
      tf.random.set_seed(42)

      tuned_model = Sequential()
      tuned_model.add(LSTM(neurons1, return_sequences=False)) 
      if dropout > 0.5:
        tuned_model.add(Dropout(dropout_rate, seed=42))
      tuned_model.add(Dense(neurons2, activation=activation))
      if dropout > 0.5:
        tuned_model.add(Dropout(dropout_rate, seed=42))
      tuned_model.add(Dense(neurons3, activation=activation))
      tuned_model.add(Dense(neurons4, activation=activation))
      tuned_model.add(Dense(1, activation='sigmoid')) #output layer

      tuned_model.compile(loss='binary_crossentropy', optimizer = optimizer, metrics=['accuracy', 'Precision', 'Recall', 'AUC'])

      return tuned_model
    
    es = EarlyStopping(monitor="loss", patience=3)
    mc = ModelCheckpoint('Best_tuned_LSTM_.h5', monitor='val_loss',mode='min',verbose=2, save_best_only=True)

    tuned_lstm = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose=1) 
    score = cross_val_score(tuned_lstm, X_train_over, y_train_over, fit_params={'callbacks':[es]}, error_score='raise', scoring='f1').mean() #5 fold cross validation default
    return score

In [ ]:
# Parameters to search
params_2 ={
    'neurons1': (128, 256),
    'neurons2': (64, 128),
    'neurons3': (32, 64),
    'neurons4': (16, 32),
    'activation':(0, 5),
    'optimizer':(0, 1),
    'learning_rate':(0.0005, 0.001),
    'batch_size':(32, 128),
    'epochs':(15, 30),
    'dropout':(0, 1),
    'dropout_rate':(0, 0.3)
}

In [ ]:
# Run Bayesian Optimization
lstm_bo_2 = BayesianOptimization(lstm_tuner_2, params_2, random_state=42)
lstm_bo_2.maximize(init_points=20, n_iter=5)

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   | learni... | neurons1  | neurons2  | neurons3  | neurons4  | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


Epoch 1/17
147/147 [==============================] - 5s 11ms/step - loss: 0.3842 - accuracy: 0.8242 - precision: 0.8080 - recall: 0.9397 - auc: 0.8932
Epoch 2/17
147/147 [==============================] - 3s 20ms/step - loss: 0.1475 - accuracy: 0.9494 - precision: 0.9444 - recall: 0.9758 - auc: 0.9802
Epoch 3/17
147/147 [==============================] - 2s 13ms/step - loss: 0.1010 - accuracy: 0.9685 - precision: 0.9615 - recall: 0.9887 - auc: 0.9884
Epoch 4/17
147/147 [==============================] - 2s 11ms/step - loss: 0.0742 - accuracy: 0.9777 - precision: 0.9728 - recall: 0.9918 - auc: 0.9928
Epoch 5/17
147/147 [==============================] - 2s 11ms/step - loss: 0.0577 - accuracy: 0.9826 - precision: 0.9787 - recall: 0.9936 - auc: 0.9953
Epoch 6/17
147/147 [==============================] - 2s 11ms/step - loss: 0.0395 - accuracy: 0.9884 - precision: 0.9853 - recall: 0.9961 - auc: 0.9975
Epoch 7/17
147/147 [==============================] - 2s 11ms/step - loss: 0.0287 - accu

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


162/162 [==============================] - 5s 11ms/step - loss: 0.3790 - accuracy: 0.8196 - precision: 0.8014 - recall: 0.9426 - auc: 0.8969
Epoch 2/18
162/162 [==============================] - 2s 11ms/step - loss: 0.1102 - accuracy: 0.9661 - precision: 0.9573 - recall: 0.9896 - auc: 0.9883
Epoch 3/18
162/162 [==============================] - 2s 11ms/step - loss: 0.0611 - accuracy: 0.9827 - precision: 0.9758 - recall: 0.9969 - auc: 0.9952
Epoch 4/18
162/162 [==============================] - 2s 11ms/step - loss: 0.0329 - accuracy: 0.9909 - precision: 0.9868 - recall: 0.9988 - auc: 0.9983
Epoch 5/18
162/162 [==============================] - 2s 12ms/step - loss: 0.0216 - accuracy: 0.9944 - precision: 0.9918 - recall: 0.9992 - auc: 0.9990
Epoch 6/18
162/162 [==============================] - 2s 11ms/step - loss: 0.0132 - accuracy: 0.9965 - precision: 0.9950 - recall: 0.9993 - auc: 0.9995
Epoch 7/18
162/162 [==============================] - 2s 11ms/step - loss: 0.0064 - accuracy: 0.998

...

145/145 [==============================] - 6s 15ms/step - loss: 0.3447 - accuracy: 0.8438 - precision: 0.8303 - recall: 0.9403 - auc: 0.9123
Epoch 2/26
145/145 [==============================] - 2s 14ms/step - loss: 0.1128 - accuracy: 0.9621 - precision: 0.9567 - recall: 0.9833 - auc: 0.9871
Epoch 3/26
145/145 [==============================] - 2s 15ms/step - loss: 0.0638 - accuracy: 0.9799 - precision: 0.9755 - recall: 0.9924 - auc: 0.9950
Epoch 4/26
145/145 [==============================] - 2s 15ms/step - loss: 0.0339 - accuracy: 0.9899 - precision: 0.9873 - recall: 0.9966 - auc: 0.9982
Epoch 5/26
145/145 [==============================] - 2s 15ms/step - loss: 0.0236 - accuracy: 0.9926 - precision: 0.9915 - recall: 0.9967 - auc: 0.9989
Epoch 6/26
145/145 [==============================] - 2s 15ms/step - loss: 0.0147 - accuracy: 0.9956 - precision: 0.9948 - recall: 0.9981 - auc: 0.9996
Epoch 7/26
145/145 [==============================] - 2s 15ms/step - loss: 0.0093 - accuracy: 0.997

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


158/158 [==============================] - 7s 15ms/step - loss: 0.3407 - accuracy: 0.8479 - precision: 0.8323 - recall: 0.9452 - auc: 0.9184
Epoch 2/28
158/158 [==============================] - 2s 15ms/step - loss: 0.1125 - accuracy: 0.9641 - precision: 0.9563 - recall: 0.9873 - auc: 0.9872
Epoch 3/28
158/158 [==============================] - 2s 15ms/step - loss: 0.0590 - accuracy: 0.9838 - precision: 0.9773 - recall: 0.9970 - auc: 0.9954
Epoch 4/28
158/158 [==============================] - 2s 14ms/step - loss: 0.0346 - accuracy: 0.9906 - precision: 0.9868 - recall: 0.9983 - auc: 0.9982
Epoch 5/28
158/158 [==============================] - 2s 14ms/step - loss: 0.0408 - accuracy: 0.9875 - precision: 0.9864 - recall: 0.9936 - auc: 0.9976
Epoch 6/28
158/158 [==============================] - 2s 14ms/step - loss: 0.0130 - accuracy: 0.9970 - precision: 0.9956 - recall: 0.9996 - auc: 0.9996
Epoch 7/28
158/158 [==============================] - 2s 14ms/step - loss: 0.0064 - accuracy: 0.998

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


181/181 [==============================] - 8s 16ms/step - loss: 0.3646 - accuracy: 0.8347 - precision: 0.8356 - recall: 0.9131 - auc: 0.9036
Epoch 2/21
181/181 [==============================] - 3s 15ms/step - loss: 0.1899 - accuracy: 0.9277 - precision: 0.9239 - recall: 0.9628 - auc: 0.9708
Epoch 3/21
181/181 [==============================] - 3s 15ms/step - loss: 0.1240 - accuracy: 0.9573 - precision: 0.9519 - recall: 0.9807 - auc: 0.9859
Epoch 4/21
181/181 [==============================] - 3s 15ms/step - loss: 0.0894 - accuracy: 0.9708 - precision: 0.9680 - recall: 0.9855 - auc: 0.9916
Epoch 5/21
181/181 [==============================] - 3s 15ms/step - loss: 0.0566 - accuracy: 0.9830 - precision: 0.9800 - recall: 0.9928 - auc: 0.9956
Epoch 6/21
181/181 [==============================] - 3s 15ms/step - loss: 0.0414 - accuracy: 0.9872 - precision: 0.9863 - recall: 0.9930 - auc: 0.9973
Epoch 7/21
181/181 [==============================] - 3s 15ms/step - loss: 0.0329 - accuracy: 0.989

In [ ]:
# Best parameters
params_lstm2_ = lstm_bo_2.max['params']
params_lstm2_

{'activation': 4.17651247794619,
 'batch_size': 62.79488623728664,
 'dropout': 0.18651851039985423,
 'dropout_rate': 0.012232542466429174,
 'epochs': 23.863394147823627,
 'learning_rate': 0.0008387821809211413,
 'neurons1': 130.12324210276557,
 'neurons2': 96.77395573115399,
 'neurons3': 39.247864806334015,
 'neurons4': 26.3227646465512,
 'optimizer': 0.17436642900499144}

In [ ]:
learning_rate = params_lstm2_['learning_rate']
activationL = ['relu', 'sigmoid', 'tanh', 'selu', 'elu', LeakyReLU]
activation= activationL[round(params_lstm2_['activation'])]
batch_size = round(params_lstm2_['batch_size'])
epochs = round(params_lstm2_['epochs'])
dropout =  params_lstm2_['dropout']
dropout_rate = params_lstm2_['dropout_rate']
neurons1 = round(params_lstm2_['neurons1'])
neurons2 = round(params_lstm2_['neurons2'])
neurons3 = round(params_lstm2_['neurons3'])
neurons4 = round(params_lstm2_['neurons4'])
optimizerL = ['Adam', 'RMSprop']
optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'RMSprop':RMSprop(learning_rate=learning_rate)}
optimizer = optimizerD[optimizerL[round(params_lstm2_['optimizer'])]]

print('Activation: ' + activation)
print('Optimizer: ' + optimizerL[round(params_lstm2_['optimizer'])])

Activation: elu
Optimizer: Adam


In [ ]:
# Best model
tuned_model_2 = Sequential()
tuned_model_2.add(LSTM(neurons1, return_sequences=False)) 
if dropout > 0.5:
  tuned_model_2.add(Dropout(dropout_rate, seed=42))
tuned_model_2.add(Dense(neurons2, activation=activation))
if dropout > 0.5:
  tuned_model_2.add(Dropout(dropout_rate, seed=42))
tuned_model_2.add(Dense(neurons3, activation=activation))
tuned_model_2.add(Dense(neurons4, activation=activation))
tuned_model_2.add(Dense(1, activation='sigmoid')) #output layer

tuned_model_2.compile(loss=BinaryCrossentropy(from_logits=False), optimizer = optimizer, metrics=['accuracy', 'Precision', 'Recall', 'AUC'])
es = EarlyStopping(monitor="loss", patience=3)
tuned_model_2.fit(X_train_over, y_train_over, batch_size=batch_size, epochs=epochs, callbacks=[es])
tuned_model_2.summary()

Epoch 1/24
359/359 [==============================] - 6s 8ms/step - loss: 0.2198 - accuracy: 0.9031 - precision: 0.8841 - recall: 0.9278 - auc: 0.9690
Epoch 2/24
359/359 [==============================] - 3s 7ms/step - loss: 0.0540 - accuracy: 0.9827 - precision: 0.9752 - recall: 0.9905 - auc: 0.9970
Epoch 3/24
359/359 [==============================] - 3s 7ms/step - loss: 0.0217 - accuracy: 0.9938 - precision: 0.9907 - recall: 0.9970 - auc: 0.9991
Epoch 4/24
359/359 [==============================] - 3s 7ms/step - loss: 0.0155 - accuracy: 0.9957 - precision: 0.9936 - recall: 0.9979 - auc: 0.9992
Epoch 5/24
359/359 [==============================] - 3s 7ms/step - loss: 0.0164 - accuracy: 0.9950 - precision: 0.9932 - recall: 0.9968 - auc: 0.9994
Epoch 6/24
359/359 [==============================] - 3s 7ms/step - loss: 0.0135 - accuracy: 0.9963 - precision: 0.9951 - recall: 0.9976 - auc: 0.9995
Epoch 7/24
359/359 [==============================] - 3s 7ms/step - loss: 0.0107 - accuracy: 0

In [ ]:
eval(tuned_model_2)

,Loss,Accuracy,Precision,Recall,AUC,F1
Train,0.000156,1.000000,1.000000,1.000000,1.000000,1.000000
Test,0.050677,0.990625,0.822222,0.660714,0.926222,0.732673


### Hyperparameter tuning 3
3 layers, diff #neurons for each

In [ ]:
def lstm_tuner_3(batch_size, neurons1, neurons2, neurons3, activation, optimizer, dropout, dropout_rate, learning_rate, epochs):

    optimizerL = ['Adam', 'RMSprop']
    optimizerD= {'Adam':Adam(learning_rate=learning_rate),'RMSprop':RMSprop(learning_rate=learning_rate)}
    activationL = ['relu', 'sigmoid', 'tanh', 'selu', 'elu', LeakyReLU]
    neurons1 = round(neurons1)
    neurons2 = round(neurons2)
    neurons3 = round(neurons3)
    activation = activationL[round(activation)]
    optimizer = optimizerD[optimizerL[round(optimizer)]]
    batch_size = round(batch_size)
    epochs = round(epochs)

    def create_model():
      # for reproducibility 
      np.random.seed(42)
      tf.random.set_seed(42)

      tuned_model = Sequential()
      tuned_model.add(LSTM(neurons1, return_sequences=False)) 
      if dropout > 0.5:
        tuned_model.add(Dropout(dropout_rate, seed=42))
      tuned_model.add(Dense(neurons2, activation=activation))
      if dropout > 0.5:
        tuned_model.add(Dropout(dropout_rate, seed=42))
      tuned_model.add(Dense(neurons3, activation=activation))
      tuned_model.add(Dense(1, activation='sigmoid')) #output layer

      tuned_model.compile(loss='binary_crossentropy', optimizer = optimizer, metrics=['accuracy', 'Precision', 'Recall', 'AUC'])

      return tuned_model
    
    es = EarlyStopping(monitor="loss", patience=3)
    mc = ModelCheckpoint('Best_tuned_LSTM_.h5', monitor='val_loss',mode='min',verbose=2, save_best_only=True)

    tuned_lstm = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose=1) 
    score = cross_val_score(tuned_lstm, X_train_over, y_train_over, fit_params={'callbacks':[es]}, error_score='raise', scoring='f1').mean() #5 fold cross validation default
    return score

In [ ]:
# Parameters to search
params_3 ={
    'neurons1': (64, 256),
    'neurons2': (32, 128),
    'neurons3': (16, 64),
    'activation':(0, 5),
    'optimizer':(0, 1),
    'learning_rate':(0.0005, 0.001),
    'batch_size':(32, 128),
    'epochs':(15, 30),
    'dropout':(0, 1),
    'dropout_rate':(0, 0.3)
}

In [ ]:
# Run Bayesian Optimization
lstm_bo_3 = BayesianOptimization(lstm_tuner_3, params_3, random_state=42)
lstm_bo_3.maximize(init_points=20, n_iter=5)

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   | learni... | neurons1  | neurons2  | neurons3  | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------
Epoch 1/17


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


147/147 [==============================] - 5s 7ms/step - loss: 0.4513 - accuracy: 0.7951 - precision: 0.7827 - recall: 0.9268 - auc: 0.8498
Epoch 2/17
147/147 [==============================] - 1s 7ms/step - loss: 0.2442 - accuracy: 0.9030 - precision: 0.9042 - recall: 0.9436 - auc: 0.9570
Epoch 3/17
147/147 [==============================] - 1s 7ms/step - loss: 0.1775 - accuracy: 0.9340 - precision: 0.9326 - recall: 0.9631 - auc: 0.9756
Epoch 4/17
147/147 [==============================] - 1s 7ms/step - loss: 0.1436 - accuracy: 0.9476 - precision: 0.9454 - recall: 0.9715 - auc: 0.9829
Epoch 5/17
147/147 [==============================] - 1s 7ms/step - loss: 0.1195 - accuracy: 0.9595 - precision: 0.9556 - recall: 0.9803 - auc: 0.9865
Epoch 6/17
147/147 [==============================] - 1s 7ms/step - loss: 0.0952 - accuracy: 0.9675 - precision: 0.9642 - recall: 0.9841 - auc: 0.9914
Epoch 7/17
147/147 [==============================] - 1s 7ms/step - loss: 0.0771 - accuracy: 0.9740 - pre

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


145/145 [==============================] - 5s 9ms/step - loss: 0.4432 - accuracy: 0.7874 - precision: 0.7647 - recall: 0.9492 - auc: 0.8588
Epoch 2/18
145/145 [==============================] - 1s 9ms/step - loss: 0.1695 - accuracy: 0.9392 - precision: 0.9334 - recall: 0.9713 - auc: 0.9777
Epoch 3/18
145/145 [==============================] - 1s 9ms/step - loss: 0.1017 - accuracy: 0.9672 - precision: 0.9595 - recall: 0.9889 - auc: 0.9899
Epoch 4/18
145/145 [==============================] - 1s 9ms/step - loss: 0.0692 - accuracy: 0.9786 - precision: 0.9729 - recall: 0.9932 - auc: 0.9947
Epoch 5/18
145/145 [==============================] - 1s 9ms/step - loss: 0.0476 - accuracy: 0.9857 - precision: 0.9814 - recall: 0.9957 - auc: 0.9971
Epoch 6/18
145/145 [==============================] - 1s 9ms/step - loss: 0.0348 - accuracy: 0.9896 - precision: 0.9865 - recall: 0.9969 - auc: 0.9981
Epoch 7/18
145/145 [==============================] - 1s 9ms/step - loss: 0.0218 - accuracy: 0.9937 - pre

...

355/355 [==============================] - 7s 7ms/step - loss: 0.2938 - accuracy: 0.8736 - precision: 0.8691 - recall: 0.9373 - auc: 0.9397
Epoch 2/28
355/355 [==============================] - 3s 7ms/step - loss: 0.1415 - accuracy: 0.9498 - precision: 0.9485 - recall: 0.9718 - auc: 0.9830
Epoch 3/28
355/355 [==============================] - 2s 7ms/step - loss: 0.0872 - accuracy: 0.9703 - precision: 0.9678 - recall: 0.9849 - auc: 0.9922
Epoch 4/28
355/355 [==============================] - 3s 7ms/step - loss: 0.0568 - accuracy: 0.9798 - precision: 0.9796 - recall: 0.9879 - auc: 0.9969
Epoch 5/28
355/355 [==============================] - 2s 7ms/step - loss: 0.0410 - accuracy: 0.9860 - precision: 0.9865 - recall: 0.9910 - auc: 0.9980
Epoch 6/28
355/355 [==============================] - 3s 7ms/step - loss: 0.0264 - accuracy: 0.9911 - precision: 0.9915 - recall: 0.9942 - auc: 0.9990
Epoch 7/28
355/355 [==============================] - 2s 7ms/step - loss: 0.0215 - accuracy: 0.9935 - pre

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


149/149 [==============================] - 7s 16ms/step - loss: 0.4533 - accuracy: 0.7980 - precision: 0.7731 - recall: 0.9543 - auc: 0.8737
Epoch 2/25
149/149 [==============================] - 2s 16ms/step - loss: 0.2316 - accuracy: 0.9193 - precision: 0.9171 - recall: 0.9563 - auc: 0.9678
Epoch 3/25
149/149 [==============================] - 2s 16ms/step - loss: 0.1684 - accuracy: 0.9490 - precision: 0.9423 - recall: 0.9776 - auc: 0.9793
Epoch 4/25
149/149 [==============================] - 2s 16ms/step - loss: 0.1290 - accuracy: 0.9614 - precision: 0.9536 - recall: 0.9856 - auc: 0.9859
Epoch 5/25
149/149 [==============================] - 2s 16ms/step - loss: 0.1105 - accuracy: 0.9671 - precision: 0.9604 - recall: 0.9876 - auc: 0.9881
Epoch 6/25
149/149 [==============================] - 2s 16ms/step - loss: 0.0850 - accuracy: 0.9764 - precision: 0.9712 - recall: 0.9913 - auc: 0.9919
Epoch 7/25
149/149 [==============================] - 2s 16ms/step - loss: 0.0745 - accuracy: 0.979

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


152/152 [==============================] - 7s 15ms/step - loss: 0.3970 - accuracy: 0.8189 - precision: 0.7922 - recall: 0.9595 - auc: 0.8959
Epoch 2/15
152/152 [==============================] - 2s 14ms/step - loss: 0.1323 - accuracy: 0.9571 - precision: 0.9470 - recall: 0.9860 - auc: 0.9851
Epoch 3/15
152/152 [==============================] - 2s 14ms/step - loss: 0.0783 - accuracy: 0.9769 - precision: 0.9691 - recall: 0.9945 - auc: 0.9932
Epoch 4/15
152/152 [==============================] - 2s 14ms/step - loss: 0.0515 - accuracy: 0.9862 - precision: 0.9797 - recall: 0.9985 - auc: 0.9967
Epoch 5/15
152/152 [==============================] - 2s 14ms/step - loss: 0.0335 - accuracy: 0.9903 - precision: 0.9856 - recall: 0.9988 - auc: 0.9985
Epoch 6/15
152/152 [==============================] - 2s 14ms/step - loss: 0.0197 - accuracy: 0.9959 - precision: 0.9934 - recall: 1.0000 - auc: 0.9992
Epoch 7/15
152/152 [==============================] - 2s 14ms/step - loss: 0.0143 - accuracy: 0.997

In [ ]:
# Best parameters
params_lstm3_ = lstm_bo_3.max['params']
params_lstm3_

{'activation': 3.0377242595071916,
 'batch_size': 48.37031587397999,
 'dropout': 0.06505159298527952,
 'dropout_rate': 0.28466566117599995,
 'epochs': 29.48448049611839,
 'learning_rate': 0.0009041986740582306,
 'neurons1': 122.48584368128718,
 'neurons2': 41.37652294461285,
 'neurons3': 48.84318527258353,
 'optimizer': 0.4401524937396013}

In [ ]:
learning_rate = params_lstm3_['learning_rate']
activationL = ['relu', 'sigmoid', 'tanh', 'selu', 'elu', LeakyReLU]
activation= activationL[round(params_lstm3_['activation'])]
batch_size = round(params_lstm3_['batch_size'])
epochs = round(params_lstm3_['epochs'])
dropout =  params_lstm3_['dropout']
dropout_rate = params_lstm3_['dropout_rate']
neurons1 = round(params_lstm3_['neurons1'])
neurons2 = round(params_lstm3_['neurons2'])
neurons3 = round(params_lstm3_['neurons3'])
optimizerL = ['Adam', 'RMSprop']
optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'RMSprop':RMSprop(learning_rate=learning_rate)}
optimizer = optimizerD[optimizerL[round(params_lstm3_['optimizer'])]]

print('Activation: ' + activation)
print('Optimizer: ' + optimizerL[round(params_lstm3_['optimizer'])])

Activation: selu
Optimizer: Adam


In [ ]:
# Best model
tuned_model_3 = Sequential()
tuned_model_3.add(LSTM(neurons1, return_sequences=False)) 
if dropout > 0.5:
  tuned_model_3.add(Dropout(dropout_rate, seed=42))
tuned_model_3.add(Dense(neurons2, activation=activation))
if dropout > 0.5:
  tuned_model_3.add(Dropout(dropout_rate, seed=42))
tuned_model_3.add(Dense(neurons3, activation=activation))
tuned_model_3.add(Dense(1, activation='sigmoid')) #output layer

tuned_model_3.compile(loss=BinaryCrossentropy(from_logits=False), optimizer = optimizer, metrics=['accuracy', 'Precision', 'Recall', 'AUC'])
es = EarlyStopping(monitor="loss", patience=3)
tuned_model_3.fit(X_train_over, y_train_over, batch_size=batch_size, epochs=epochs, callbacks=[es])
tuned_model_3.summary()

Epoch 1/29
471/471 [==============================] - 7s 6ms/step - loss: 0.2150 - accuracy: 0.9106 - precision: 0.8973 - recall: 0.9272 - auc: 0.9712
Epoch 2/29
471/471 [==============================] - 3s 6ms/step - loss: 0.0547 - accuracy: 0.9825 - precision: 0.9739 - recall: 0.9915 - auc: 0.9969
Epoch 3/29
471/471 [==============================] - 3s 6ms/step - loss: 0.0277 - accuracy: 0.9915 - precision: 0.9882 - recall: 0.9949 - auc: 0.9989
Epoch 4/29
471/471 [==============================] - 3s 7ms/step - loss: 0.0119 - accuracy: 0.9964 - precision: 0.9945 - recall: 0.9983 - auc: 0.9996
Epoch 5/29
471/471 [==============================] - 3s 6ms/step - loss: 0.0161 - accuracy: 0.9947 - precision: 0.9930 - recall: 0.9965 - auc: 0.9994
Epoch 6/29
471/471 [==============================] - 3s 6ms/step - loss: 0.0114 - accuracy: 0.9969 - precision: 0.9960 - recall: 0.9977 - auc: 0.9994
Epoch 7/29
471/471 [==============================] - 3s 6ms/step - loss: 0.0036 - accuracy: 0

In [ ]:
eval(tuned_model_3)

,Loss,Accuracy,Precision,Recall,AUC,F1
Train,5.782691e-07,1.000000,1.000000,1.000000,1.000000,1.00
Test,7.948051e-02,0.990972,0.840909,0.660714,0.864535,0.74
